In [ ]:
from DeepPurpose import DTI
from DeepPurpose.dataset import *

from sklearn import metrics
import pandas as pd

In [ ]:
# Define the path where the KIBA dataset is saved
SAVE_PATH = '../data/KIBA'

In [ ]:
# Load and process the KIBA dataset
X_drug, X_target, y = load_process_KIBA(SAVE_PATH, binary=False)

In [ ]:
len(X_drug), len(set(X_drug))

In [ ]:
len(X_target), len(set(X_target))

In [ ]:
# Specify the encoding methods for drugs and targets
drug_encoding, target_encoding = 'MPNN', 'CNN'

In [ ]:
# Process the dataset using the specified encoding methods without splitting it
dataset = data_process(X_drug, X_target, y, drug_encoding, target_encoding, split_method='no_split')

In [ ]:
# Read and preprocess the affinity data, filling missing values with -1
affinity = pd.read_csv(SAVE_PATH + '/KIBA/affinity.txt', header=None, sep = '\t')
affinity = affinity.fillna(-1)

# Load target sequences and drug SMILES from the files
with open(SAVE_PATH + '/KIBA/target_seq.txt') as f:
    target = json.load(f)

with open(SAVE_PATH + '/KIBA/SMILES.txt') as f:
    drug = json.load(f)

target = list(target)
drug = list(drug)

drug_ids = []
target_ids = []

# Iterate over drug-target pairs, recording identifiers where affinity is present
for i in range(len(drug)):
    for j in range(len(target)):
        if affinity.values[i, j] != -1:
            drug_ids.append(drug[i])
            target_ids.append(target[j])

In [ ]:
# Add drug and target ids to the dataset
dataset['drug_id'] = drug_ids
dataset['target_id'] = target_ids

In [ ]:
# Load a pre-trained DTI model 
model = DTI.model_pretrained(model='mpnn_cnn_kiba')

In [ ]:
# Predict the interactions using the model and insert the predictions into the dataset
predicted = model.predict(dataset)
dataset.insert(3, 'predicted', predicted)

In [ ]:
dataset[['drug_id', 'target_id', 'Label', 'predicted']]

In [ ]:
# Save the drug-target pairs with actual and predicted labels to a CSV file
dataset[['drug_id', 'target_id', 'Label', 'predicted']].to_csv('../analysis/mpnn_cnn_kiba_predictions.csv', index=False)

In [ ]:
# Calculate the R-squared score and RMSE between actual and predicted labels
r2_score = metrics.r2_score(dataset.Label, dataset.predicted)
r2_score

In [ ]:
mse = metrics.mean_squared_error(dataset.Label, dataset.predicted, squared=False)
mse

In [ ]:
metrics_df = pd.DataFrame({
    'Metric': ['R-squared', 'MSE'],
    'Value_general': [r2_score, mse]
})
metrics_df

In [ ]:
metrics_df.to_csv('../analysis/mpnn_cnn_kiba_performance.csv', index=False)